<a href="https://colab.research.google.com/github/SpirinEgor/topic_modeling_story/blob/master/train_ctm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Tue Nov 30 20:41:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    74W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! git clone https://github.com/SpirinEgor/topic_modeling_story
%cd topic_modeling_story

fatal: destination path 'topic_modeling_story' already exists and is not an empty directory.
/content/topic_modeling_story


In [4]:
! git pull

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7 (delta 4), reused 7 (delta 4), pack-reused 0
Unpacking objects: 100% (7/7), done.
From https://github.com/SpirinEgor/topic_modeling_story
   6c8c346..d666276  master     -> origin/master
Updating 6c8c346..d666276
Fast-forward
 src/ctm.py     |  8 ++++----
 src/dataset.py | 15 +++++++++------
 2 files changed, 13 insertions(+), 10 deletions(-)


In [4]:
! wget https://voudy-data.s3.eu-north-1.amazonaws.com/pycharm_issues.json

--2021-11-30 20:03:03--  https://voudy-data.s3.eu-north-1.amazonaws.com/pycharm_issues.json
Resolving voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)... 52.95.170.44
Connecting to voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)|52.95.170.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27987977 (27M) [application/json]
Saving to: ‘pycharm_issues.json’

pycharm_issues.json 100%[===================>]  26.69M  54.7MB/s    in 0.5s    

2021-11-30 20:03:03 (54.7 MB/s) - ‘pycharm_issues.json’ saved [27987977/27987977]



In [5]:
%%capture
! pip install contextualized-topic-models pyldavis

In [5]:
import pickle

import nltk
import pyLDAvis as vis

from src.dataset import YouTrackIssueDataset
from src.ctm import ContextualizedTopicModeling

%matplotlib inline
%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [6]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
dataset = YouTrackIssueDataset("pycharm_issues.json")
len(dataset)

21832

In [8]:
%%time
ctm = ContextualizedTopicModeling(dataset, "paraphrase-distilroberta-base-v2")

Found 21832 documents
Preprocessing documents...


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Done in 0.9311788082122803 s


Batches:   0%|          | 0/109 [00:00<?, ?it/s]

CPU times: user 45.7 s, sys: 1.69 s, total: 47.3 s
Wall time: 51.8 s


In [9]:
ctm.fit(contextual_size=768, n_topics=100, num_epochs=50)

Epoch: [50/50]	 Seen Samples: [1089400/1089400]	Train Loss: 76.93202032709375	Time: 0:00:09.271805: : 50it [07:40,  9.22s/it]
Sampling: [20/20]: : 20it [02:38,  7.90s/it]


In [10]:
ctm.save("cmt_model.pkl")

In [6]:
ctm = ContextualizedTopicModeling.load("cmt_model.pkl")

In [13]:
lda_vis_data = ctm.model.get_ldavis_data_format(ctm.tp.vocab, ctm.train_dataset, 20)
vis_pd = vis.prepare(**lda_vis_data)
with open("lda_vis_data.pkl", "wb") as f:
    pickle.dump(vis_pd, f)

Sampling: [20/20]: : 20it [02:38,  7.91s/it]


In [14]:
vis.display(vis_pd)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [17]:
summaries = [dataset[i][0] for i in range(len(dataset))]
transformed = ctm.tp.transform(summaries)
topics_predictions = ctm.model.get_thetas(transformed, n_samples=20)  # get all the topic predictions
with open("predictions.pkl", "wb") as f:
    pickle.dump(topics_predictions, f)

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/utils/data_preparation.py:98: UserWarning: The method did not have in input the text_for_bow parameter. This IS EXPECTED if you are using ZeroShotTM in a cross-lingual setting
  warnings.warn("The method did not have in input the text_for_bow parameter. This IS EXPECTED if you "


Batches:   0%|          | 0/110 [00:00<?, ?it/s]

Sampling: [1/20]: : 1it [17:55, 1075.18s/it]
Sampling: [20/20]: : 20it [02:37,  7.87s/it]


In [19]:
from google.colab import files

files.download("lda_vis_data.pkl")
files.download("predictions.pkl")
files.download("cmt_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>